# Entrenamiento del modelo
## Modificación en las columnas

Creación de nuevas columnas para agregar a la transacción información historica del gasto del cliente.

Eliminación de las columnas irrelevantes para la clasificación de la transacción.

In [1]:
import pandas as pd
import numpy as np

names = ['id_trn_ach', 'id_cliente', 'fecha', 'hora', 'valor_trx', 'class', 'seg_str', 'ocupacion', 'tipo_vivienda', 'nivel_academico', 'estado_civil', 'genero', 'edad', 'ingreso_rango']
per = pd.read_csv('personas.csv', names=['id_cliente', 'seg_str', 'ocupacion', 'tipo_vivienda', 'nivel_academico', 'estado_civil', 'genero', 'edad', 'ingreso_rango'])
trx = pd.read_csv('set4.csv', names=['id_trn_ach', 'id_cliente', 'fecha', 'hora', 'valor_trx', 'class'])

In [2]:
#df.head
print('Filas: {}\nColumnas: {}\n'.format(per.shape[0], per.shape[1]))
print('Filas: {}\nColumnas: {}\n'.format(trx.shape[0], trx.shape[1]))
print(per.isnull().sum())

Filas: 338606
Columnas: 9

Filas: 7490652
Columnas: 6

id_cliente              0
seg_str                 0
ocupacion            6837
tipo_vivienda      184383
nivel_academico     93650
estado_civil        67360
genero               5709
edad                 8216
ingreso_rango        7384
dtype: int64


In [3]:
df = pd.merge(trx, per, on=['id_cliente','id_cliente'])
print('Filas: {}\nColumnas: {}\n'.format(df.shape[0], df.shape[1]))
print(df.isnull().sum())

Filas: 7490652
Columnas: 14

id_trn_ach               0
id_cliente               0
fecha                    0
hora                     0
valor_trx                0
class                    0
seg_str                  0
ocupacion           137927
tipo_vivienda      3230964
nivel_academico    1324842
estado_civil       1381618
genero              127532
edad                165074
ingreso_rango        59925
dtype: int64


In [ ]:
df.to_csv('set3.csv', header=False, index=False)

In [ ]:
from tqdm import tqdm

client_trx = dict()
clientes = df['id_cliente'].unique()
for client in tqdm(clientes):
    if client not in client_trx:
        client_trx[client] = df.loc[df['id_cliente'] == client]

In [ ]:
val = list()
nan = np.nan
for id_cliente, fecha in tqdm(zip(df['id_cliente'],df['fecha'])):
    tmp = client_trx[id_cliente][client_trx[id_cliente]['fecha'] < fecha].tail(10)    
    if len(tmp) > 0: 
        val.append(tmp['class'].mode()[0])
    else:
        val.append(nan)

In [ ]:
df = df.assign(last_class=val)
df.to_csv('set4_1.csv',header=False)

In [ ]:
from datetime import datetime
from datetime import timedelta

val = list()
nan = np.nan
for id_cliente, fecha in tqdm(zip(df['id_cliente'],df['fecha'])):
    tmp = client_trx[id_cliente][client_trx[id_cliente]['fecha'] < fecha]
    if len(tmp) > 0:
        _fecha = datetime.strptime(str(int(fecha)), '%Y%m%d') - timedelta(365/12)        
        _fecha = int(_fecha.strftime('%Y%m%d'))
        idx_val = tmp['fecha'].searchsorted(value=_fecha)[0]
        if idx_val >= len(tmp): idx_val -= 1
        _delta = datetime.strptime(str(int(fecha)), '%Y%m%d') - datetime.strptime(str(int(tmp.iloc[idx_val]['fecha'])), '%Y%m%d')
        _delta = _delta.days
        if _delta > 0: val.append(tmp.iloc[idx_val]['class'])
    else:
        val.append(nan)

In [ ]:
df = df.assign(last_month=val)
df.to_csv('set4_2.csv',header=False)

In [4]:
# separar fecha en mes y dia, eliminar año
from tqdm import tqdm
from datetime import datetime

fecha = df['fecha']
mes, dia = list(), list()
for f in tqdm(fecha):
    f = datetime.strptime(str(f), '%Y%m%d')
    mes.append(f.month)
    dia.append(f.day)

100%|██████████| 7490652/7490652 [01:13<00:00, 101282.87it/s]


In [5]:
# eliminar fecha, id trx, id cliente y agregar mes y dia
df = df.assign(mes=mes)
df = df.assign(dia=dia)
df = df.drop('id_trn_ach', axis = 1)
df = df.drop('id_cliente', axis = 1)
df = df.drop('fecha', axis = 1)

In [ ]:
df.to_csv('set5.csv', header=False, index=False)